### Abdelrahman Alkhawas Sprint 3
This is my sprint two notebook where I will be Reconfiguring my data for some initial NN experimentation. This notebook assumes that the user has access to `eeaao.csv` or has run the latest iteration of `webScraping1.py`.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Masking, LSTM, Dense

pd.options.display.max_columns = None
pd.options.display.max_rows = 200
pd.set_option('display.width', 1000)

In [2]:
# Importing my Data. 
df = pd.read_csv('../data/eeaao.csv')
df = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
df.head()

,p_id,Season,Player,Nation,Pos,Squad,Comp,Age,Born,"Playing Time, MP","Playing Time, Starts","Playing Time, Min","Playing Time, 90s","Performance, Gls","Performance, Ast","Performance, G+A","Performance, G-PK","Performance, PK","Performance, PKatt","Performance, CrdY","Performance, CrdR","Expected, xG","Expected, npxG","Expected, xAG","Expected, npxG+xAG","Progression, PrgC","Progression, PrgP","Progression, PrgR","Per 90 Minutes, Gls","Per 90 Minutes, Ast","Per 90 Minutes, G+A","Per 90 Minutes, G-PK","Per 90 Minutes, G+A-PK","Per 90 Minutes, xG","Per 90 Minutes, xAG","Per 90 Minutes, xG+xAG","Per 90 Minutes, npxG","Per 90 Minutes, npxG+xAG","Playing Time, MP.1","Playing Time, Min.1","Playing Time, Mn/MP","Playing Time, Min%","Playing Time, 90s.1","Starts, Starts","Starts, Mn/Start","Starts, Compl","Subs, Subs","Subs, Mn/Sub","Subs, unSub","Team Success, PPM","Team Success, onG","Team Success, onGA","Team Success, +/-","Team Success, +/-90","Team Success, On-Off","Team Success (xG), onxG","Team Success (xG), onxGA","Team Success (xG), xG+/-","Team Success (xG), xG+/-90","Team Success (xG), On-Off","Standard, Gls","Standard, Sh","Standard, SoT","Standard, SoT%","Standard, Sh/90","Standard, SoT/90","Standard, G/Sh","Standard, G/SoT","Standard, Dist","Standard, FK","Standard, PK","Standard, PKatt","Expected, xG.1","Expected, npxG.1","Expected, npxG/Sh","Expected, G-xG","Expected, np:G-xG","Total, Cmp","Total, Att","Total, Cmp%","Total, TotDist","Total, PrgDist","Short, Cmp","Short, Att","Short, Cmp%","Medium, Cmp","Medium, Att","Medium, Cmp%","Long, Cmp","Long, Att","Long, Cmp%",Ast,xAG,"Expected, xA","Expected, A-xAG",KP,1/3,PPA,CrsPA,PrgP,Att,"Pass Types, Live","Pass Types, Dead","Pass Types, FK","Pass Types, TB","Pass Types, Sw","Pass Types, Crs","Pass Types, TI","Pass Types, CK","Corner Kicks, In","Corner Kicks, Out","Corner Kicks, Str","Outcomes, Cmp","Outcomes, Off","Outcomes, Blocks","SCA, SCA","SCA, SCA90","SCA Types, PassLive","SCA Types, PassDead","SCA Types, TO","SCA Types, Sh","SCA Types, Fld","SCA Types, Def","GCA, GCA","GCA, GCA90","GCA Types, PassLive","GCA Types, PassDead","GCA Types, TO","GCA Types, Sh","GCA Types, Fld","GCA Types, Def","Tackles, Tkl","Tackles, TklW","Tackles, Def 3rd","Tackles, Mid 3rd","Tackles, Att 3rd","Challenges, Tkl","Challenges, Att","Challenges, Tkl%","Challenges, Lost","Blocks, Blocks","Blocks, Sh","Blocks, Pass",Int,Tkl+Int,Clr,Err,"Touches, Touches","Touches, Def Pen","Touches, Def 3rd","Touches, Mid 3rd","Touches, Att 3rd","Touches, Att Pen","Touches, Live","Take-Ons, Att","Take-Ons, Succ","Take-Ons, Succ%","Take-Ons, Tkld","Take-Ons, Tkld%","Carries, Carries","Carries, TotDist","Carries, PrgDist","Carries, PrgC","Carries, 1/3","Carries, CPA","Carries, Mis","Carries, Dis","Receiving, Rec","Receiving, PrgR"
0,Patrick van Aanholt-NED-1990,2017-2018,Patrick van Aanholt,nl NED,1.0,Crystal Palace,eng Premier League,26,1990,28,25,2184,24.3,5,1,6,5,0,0,7,0,3.1,3.1,2.1,5.2,46.0,92.0,86.0,0.21,0.04,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,28,2184,78,63.9,24.3,25,NaN,21.0,3,NaN,8,1.21,31.0,38.0,-7.0,-0.29,-0.07,32.2,34.7,-2.5,-0.10,-0.66,5,33.0,11,33.3,1.36,0.45,0.15,0.45,23.4,4.0,0,0,3.1,3.1,0.09,1.9,1.9,884.0,1176.0,75.2,14197.0,6422.0,479.0,532.0,90.0,311.0,431.0,72.2,74.0,152.0,48.7,1,2.1,1.8,-1.1,18.0,63.0,28.0,6.0,92.0,1176.0,897.0,276.0,30.0,3.0,2.0,56.0,235.0,11.0,3.0,5.0,2.0,884.0,3.0,31.0,46.0,1.90,24.0,12.0,6.0,2.0,1.0,1.0,4.0,0.16,2.0,0.0,2.0,0.0,0.0,0.0,47.0,32.0,29.0,15.0,3.0,16.0,34.0,47.1,18.0,24.0,5.0,19.0,47.0,94.0,64.0,2.0,1435.0,90.0,445.0,616.0,391.0,32.0,1435.0,31.0,26.0,83.9,5.0,16.1,717.0,3714.0,1966.0,46.0,33.0,7.0,37.0,18.0,711.0,86.0
1,Rolando Aarons-ENG-1995,2017-2018,Rolando Aarons,eng ENG,5.0,Newcastle Utd,eng Premier League,21,1995,4,1,139,1.5,0,0,0,0,0,0,0,0,0.1,0.1,0.0,0.1,7.0,3.0,4.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,4,139,35,4.1,1.5,1,NaN,0.0,3,NaN,7,0.25,1.0,2.0,-1.0,-0.65,-0.46,0.4,2.9,-2.6,-1.66,-1.43,0,2.0,0,0.0,1.29,0.00,0.00,NaN,19.9,0.0,

In [3]:
for col in df.columns: print(col)

p_id
Season
Player
Nation
Pos
Squad
Comp
Age
Born
Playing Time, MP
Playing Time, Starts
Playing Time, Min
Playing Time, 90s
Performance, Gls
Performance, Ast
Performance, G+A
Performance, G-PK
Performance, PK
Performance, PKatt
Performance, CrdY
Performance, CrdR
Expected, xG
Expected, npxG
Expected, xAG
Expected, npxG+xAG
Progression, PrgC
Progression, PrgP
Progression, PrgR
Per 90 Minutes, Gls
Per 90 Minutes, Ast
Per 90 Minutes, G+A
Per 90 Minutes, G-PK
Per 90 Minutes, G+A-PK
Per 90 Minutes, xG
Per 90 Minutes, xAG
Per 90 Minutes, xG+xAG
Per 90 Minutes, npxG
Per 90 Minutes, npxG+xAG
Playing Time, MP.1
Playing Time, Min.1
Playing Time, Mn/MP
Playing Time, Min%
Playing Time, 90s.1
Starts, Starts
Starts, Mn/Start
Starts, Compl
Subs, Subs
Subs, Mn/Sub
Subs, unSub
Team Success, PPM
Team Success, onG
Team Success, onGA
Team Success, +/-
Team Success, +/-90
Team Success, On-Off
Team Success (xG), onxG
Team Success (xG), onxGA
Team Success (xG), xG+/-
Team Success (xG), xG+/-90
Team Success (

#### Some initial thoughts
Following are some initial thoughts I was planning on tackling this problem that couldnt take off the ground, and whose EDA/planning was too messy/incoherent to make it to this notebook.
<br><br>
I was looking to treat each datapoint as a sentence in typical RNN fashion. This would skip the usual token vectorization since the data is already numeric. However, now comes the question of 'what is a datapoint?' for any given player, do we have 7 datapoints (one for each season) of sentence length 160 and we try to predict the entire next year, or do we have 160 datapoints (one for each feature) of size 7 and we try to predict the next element of that features sequence?
<br>
The issue with the first way is that sentences are extremely long and the idea of generating sentences that way can lead to 'jumbled sentenced'. The issue with the second way is that we have to teach 160 models how to interact with each other and we lose any value of using NNs due to lack of interaction of features. 
<br><br>
Whilst the second approach makes no sense to use, the first approach has somoe promise, with some alterations to the structure of the data and the preprocessing techniques. 

---

#### The approach
1. We will runn a similar 'cleaning' operation on the entire dataset to mae sure each player has only one datapoint per season
2. For our first attempt we will also filter to only players that have datapoints for every season (so we dont have to worry about putting NaNs into the model) and only try to predict our previously defined 'impact' metrics
3. Generate the cleaned 3D matrix and to a traintest separate X_train and Y_train
4. train a simple (not too many layers/ big layers) LTSM NN 
5. evaluate
##### Problems with Approach 1
- Missing important players (obv)
- Bias towards aging players or players on teams that stay up
##### Places to progress
- train on everything. This will require different preprocessing
- train on all players. Will also require funky preprocessing.
- bigger NN
- different activations
- combine all the above. 


In [4]:
# The objective of this block of code is to consolidate player datapoints such that each player for each season has 'at most' one row. 
# We are also not dealing with any funny business with transfers.

# For this, we only need the following columns:
tdf = df.loc[:, ['p_id', 'Player', 'Pos', 'Age', 'Playing Time, Min', 'Team Success, onGA', 'Team Success, onG', 'Starts, Starts', 'Season', 'Squad']]
tgb = df.groupby('p_id')  # Group by the unique players. 

temp_dict = {}  # {pid: df}
for key in tgb.groups.keys():
    if tgb.get_group(key).shape[0] >= 6:
        temp_dict[key] = tgb.get_group(key)

full_careers = {}  # {pid: cleaned_df}
total = pd.DataFrame()
n_full = 0

# want to get rid of rows where trnsfer window gets in the way. This means for the same season, there are two rows
# If for one of them has Playing Time, Min > 180, ignore this player. Otherwise, take the max. 
for pid in temp_dict.keys():
    print(pid)
    omit_player = False
    # gor into pdf and ensure for each 
    # go through each season JUST TO CHECK WHETHER WE SHOULD OMIT THS PLAYER
    temp_df_gb = temp_dict[pid].groupby('Season')  # groupby object where keys are seasons
    for season in temp_df_gb.groups.keys(): 
        season_df = temp_df_gb.get_group(season)
        if season_df.shape[0] >= 2 and sum(season_df['Playing Time, Min'] > 270) >= 2:  
            # For this season, there are two entries, both of which have too many minutes and we should omit this entry
            omit_player = True
            temp_dict[key] = tgb.get_group(key)

    full_careers[pid] = temp_dict[pid].loc[temp_df_gb['Playing Time, Min'].idxmax()]
    if not omit_player and full_careers[pid].shape[0] == 7:
        n_full+= 1
        print(n_full)
        total = pd.concat([total, full_careers[pid]], axis=0)

# Now full careers contans all of the players who have at least 7 datapoints {p_id: df}  717 of them
print(total.shape)

Aaron Cresswell-ENG-1989
1
Aaron Ramsey-WAL-1990
Aaron Wan-Bissaka-ENG-1997
2
Aarón Martín-ESP-1997
3
Abdou Diallo-SEN-1996
Abdoulaye Bamba-CIV-1990
Abdoulaye Doucouré-MLI-1993
4
Abdoulaye Touré-GUI-1994
Achraf Hakimi-MAR-1998
5
Adam Lallana-ENG-1988
6
Adam Marušić-MNE-1992
7
Adam Masina-MAR-1994
Adam Ounas-ALG-1996
Adama Soumaoro-FRA-1992
Adama Traoré-ESP-1996
Ademola Lookman-NGA-1997
8
Admir Mehmedi-SUI-1991
Adnan Januzaj-BEL-1995
9
Adri Embarba-ESP-1992
Adrien Rabiot-FRA-1995
10
Adrien Silva-POR-1989
Adrien Tameze-CMR-1994
11
Adrien Thomasson-FRA-1993
Aihen Muñoz-ESP-1997
Ainsley Maitland-Niles-ENG-1997
Aitor Fernández-ESP-1991
Aitor Ruibal-ESP-1996
Alassane Pléa-FRA-1993
12
Alban Lafont-FRA-1999
13
Alberto Grassi-ITA-1995
14
Alberto Moreno-ESP-1992
15
Albin Ekdal-SWE-1989
Aleix Vidal-ESP-1989
Aleksandr Golovin-RUS-1996
Alessandro Bastoni-ITA-1999
16
Alessandro Deiola-ITA-1995
Alessandro Florenzi-ITA-1991
Alessio Cragno-ITA-1994
17
Alessio Romagnoli-ITA-1995
18
Alex Ferrari-ITA-1994

In [5]:
total.head()

,p_id,Season,Player,Nation,Pos,Squad,Comp,Age,Born,"Playing Time, MP","Playing Time, Starts","Playing Time, Min","Playing Time, 90s","Performance, Gls","Performance, Ast","Performance, G+A","Performance, G-PK","Performance, PK","Performance, PKatt","Performance, CrdY","Performance, CrdR","Expected, xG","Expected, npxG","Expected, xAG","Expected, npxG+xAG","Progression, PrgC","Progression, PrgP","Progression, PrgR","Per 90 Minutes, Gls","Per 90 Minutes, Ast","Per 90 Minutes, G+A","Per 90 Minutes, G-PK","Per 90 Minutes, G+A-PK","Per 90 Minutes, xG","Per 90 Minutes, xAG","Per 90 Minutes, xG+xAG","Per 90 Minutes, npxG","Per 90 Minutes, npxG+xAG","Playing Time, MP.1","Playing Time, Min.1","Playing Time, Mn/MP","Playing Time, Min%","Playing Time, 90s.1","Starts, Starts","Starts, Mn/Start","Starts, Compl","Subs, Subs","Subs, Mn/Sub","Subs, unSub","Team Success, PPM","Team Success, onG","Team Success, onGA","Team Success, +/-","Team Success, +/-90","Team Success, On-Off","Team Success (xG), onxG","Team Success (xG), onxGA","Team Success (xG), xG+/-","Team Success (xG), xG+/-90","Team Success (xG), On-Off","Standard, Gls","Standard, Sh","Standard, SoT","Standard, SoT%","Standard, Sh/90","Standard, SoT/90","Standard, G/Sh","Standard, G/SoT","Standard, Dist","Standard, FK","Standard, PK","Standard, PKatt","Expected, xG.1","Expected, npxG.1","Expected, npxG/Sh","Expected, G-xG","Expected, np:G-xG","Total, Cmp","Total, Att","Total, Cmp%","Total, TotDist","Total, PrgDist","Short, Cmp","Short, Att","Short, Cmp%","Medium, Cmp","Medium, Att","Medium, Cmp%","Long, Cmp","Long, Att","Long, Cmp%",Ast,xAG,"Expected, xA","Expected, A-xAG",KP,1/3,PPA,CrsPA,PrgP,Att,"Pass Types, Live","Pass Types, Dead","Pass Types, FK","Pass Types, TB","Pass Types, Sw","Pass Types, Crs","Pass Types, TI","Pass Types, CK","Corner Kicks, In","Corner Kicks, Out","Corner Kicks, Str","Outcomes, Cmp","Outcomes, Off","Outcomes, Blocks","SCA, SCA","SCA, SCA90","SCA Types, PassLive","SCA Types, PassDead","SCA Types, TO","SCA Types, Sh","SCA Types, Fld","SCA Types, Def","GCA, GCA","GCA, GCA90","GCA Types, PassLive","GCA Types, PassDead","GCA Types, TO","GCA Types, Sh","GCA Types, Fld","GCA Types, Def","Tackles, Tkl","Tackles, TklW","Tackles, Def 3rd","Tackles, Mid 3rd","Tackles, Att 3rd","Challenges, Tkl","Challenges, Att","Challenges, Tkl%","Challenges, Lost","Blocks, Blocks","Blocks, Sh","Blocks, Pass",Int,Tkl+Int,Clr,Err,"Touches, Touches","Touches, Def Pen","Touches, Def 3rd","Touches, Mid 3rd","Touches, Att 3rd","Touches, Att Pen","Touches, Live","Take-Ons, Att","Take-Ons, Succ","Take-Ons, Succ%","Take-Ons, Tkld","Take-Ons, Tkld%","Carries, Carries","Carries, TotDist","Carries, PrgDist","Carries, PrgC","Carries, 1/3","Carries, CPA","Carries, Mis","Carries, Dis","Receiving, Rec","Receiving, PrgR"
575,Aaron Cresswell-ENG-1989,2017-2018,Aaron Cresswell,eng ENG,1.0,West Ham,eng Premier League,27,1989,36,35,3069,34.1,1,3,4,1,0,0,7,0,0.8,0.8,3.2,3.9,35.0,128.0,62.0,0.03,0.09,0.12,0.03,0.12,0.02,0.09,0.12,0.02,0.12,36,3069,85,89.7,34.1,35,NaN,30.0,1,NaN,1,1.14,45.0,60.0,-15.0,-0.44,0.84,34.5,47.8,-13.3,-0.39,0.92,1,21.0,6,28.6,0.62,0.18,0.05,0.17,28.7,8.0,0,0,0.8,0.8,0.04,0.2,0.2,1204.0,1723.0,69.9,22518.0,9345.0,531.0,593.0,89.5,488.0,642.0,76.0,163.0,408.0,40.0,3,3.2,2.4,-0.2,34.0,135.0,21.0,10.0,128.0,1723.0,1346.0,361.0,82.0,3.0,23.0,196.0,212.0,67.0,43.0,14.0,2.0,1204.0,16.0,51.0,66.0,1.94,36.0,24.0,1.0,2.0,3.0,0.0,9.0,0.26,6.0,3.0,0.0,0.0,0.0,0.0,34.0,21.0,12.0,20.0,2.0,19.0,30.0,63.3,11.0,42.0,17.0,25.0,44.0,78.0,117.0,1.0,2035.0,120.0,572.0,982.0,497.0,15.0,2035.0,14.0,5.0,35.7,9.0,64.3,870.0,4032.0,1959.0,35.0,48.0,1.0,26.0,20.0,1076.0,62.0
3230,Aaron Cresswell-ENG-1989,2018-2019,Aaron Cresswell,eng ENG,1.0,West Ham,eng Premier League,28,1989,20,18,1589,17.7,0,1,1,0,0,0,1,0,0.5,0.5,1.1,1.5,35.0,94.0,103.0,0.00,0.06,0.06,0.00,0.06,0.03,0.06,0.09,0.03,0.09,20,1589,79,46.5,17.7,18,85.0,16.0,2,30.0,7,1.30,21.0,26.0,-5.0,-0.28,-0.38,19.7,25.2,-5.5,-0.31,0.12,0,11.0,0,0.0,0.62

In [6]:
type(total['Subs, Mn/Sub'][575])

numpy.float64

In [50]:
total.sort_values(['p_id', 'Season'])

placeholder_value = -999.0
nan_total = total.fillna(placeholder_value)

# We now need to set up the 3d matrix and do a train/test split

players = total['p_id'].unique()

# Reserving a small portion of the players for testing
train_players, test_players = train_test_split(players, test_size=0.1, random_state=42)  

# Now put the portions of the players into the 3d format
train_indices = [i for i, player in enumerate(players) if player in train_players]
test_indices = [i for i, player in enumerate(players) if player in test_players]

# Get from total
data = nan_total.to_numpy().reshape(496, 7, 169)

# getting non-string features
i_s = [4]
i_s.extend(list(range(7, 169)))
data = data[:, :, i_s]

X_train, Y_train = data[train_indices, :-1, :], data[train_indices, -1, :]
X_test, Y_test = data[test_indices, :-1, :], data[test_indices, -1, :]
X_train = X_train.astype(np.float32)
Y_train = Y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
Y_test = Y_test.astype(np.float32)

In [47]:
# Print shapes for verification
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)
print(X_train.dtype)

X_train shape: (446, 6, 163)
Y_train shape: (446, 163)
X_test shape: (50, 6, 163)
Y_test shape: (50, 163)
float32


In [ ]:
# Modelling
# Create initial model
model = keras.Sequential()

# Add just one layer for now
model.add(Masking(mask_value=placeholder_value))
model.add(layers.LSTM(256, activation='tanh', return_sequences=True))
model.add(layers.LSTM(512, activation='tanh', return_sequences=True))
model.add(layers.LSTM(, activation='tanh', return_sequences=True))
model.add(layers.LSTM(128, activation='tanh', return_sequences=False))

# Output layer
model.add(layers.Dense(X_train.shape[2], activation='linear')) # output layer has ~160 linear nodes

# Compile model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
              loss=keras.losses.MeanSquaredError(), 
              metrics=[keras.metrics.R2Score()])

# Define an Early Stop callback
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience = 10, verbose=1)

# fit the model
history = model.fit(X_train, Y_train, batch_size=128, epochs=500, verbose=1)

model.evaluate(X_test, Y_test, verbose=0)


Epoch 1/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - loss: 2443047.0000 - r2_score: -2227.5093
Epoch 2/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2322819.0000 - r2_score: -2143.2524
Epoch 3/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2306790.0000 - r2_score: -2495.5679
Epoch 4/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2229660.5000 - r2_score: -2178.8955
Epoch 5/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2246287.0000 - r2_score: -2119.8931
Epoch 6/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2473057.0000 - r2_score: -2080.0022
Epoch 7/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 2169462.5000 - r2_score: -2113.6904
Epoch 8/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2186493.7500 - r2_score: -2195.6611
Epoch 9/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2178917.5000 - r2_score: -2353.0615
Epoch 10/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2144818.5000 - r2_score: -2266.8748
Epoch 11/200
28/28 ━━━━━━━━━━━━━━━━━━━━

[1552297.0, -2145.04052734375]

In [49]:
nan_total

,p_id,Season,Player,Nation,Pos,Squad,Comp,Age,Born,"Playing Time, MP","Playing Time, Starts","Playing Time, Min","Playing Time, 90s","Performance, Gls","Performance, Ast","Performance, G+A","Performance, G-PK","Performance, PK","Performance, PKatt","Performance, CrdY","Performance, CrdR","Expected, xG","Expected, npxG","Expected, xAG","Expected, npxG+xAG","Progression, PrgC","Progression, PrgP","Progression, PrgR","Per 90 Minutes, Gls","Per 90 Minutes, Ast","Per 90 Minutes, G+A","Per 90 Minutes, G-PK","Per 90 Minutes, G+A-PK","Per 90 Minutes, xG","Per 90 Minutes, xAG","Per 90 Minutes, xG+xAG","Per 90 Minutes, npxG","Per 90 Minutes, npxG+xAG","Playing Time, MP.1","Playing Time, Min.1","Playing Time, Mn/MP","Playing Time, Min%","Playing Time, 90s.1","Starts, Starts","Starts, Mn/Start","Starts, Compl","Subs, Subs","Subs, Mn/Sub","Subs, unSub","Team Success, PPM","Team Success, onG","Team Success, onGA","Team Success, +/-","Team Success, +/-90","Team Success, On-Off","Team Success (xG), onxG","Team Success (xG), onxGA","Team Success (xG), xG+/-","Team Success (xG), xG+/-90","Team Success (xG), On-Off","Standard, Gls","Standard, Sh","Standard, SoT","Standard, SoT%","Standard, Sh/90","Standard, SoT/90","Standard, G/Sh","Standard, G/SoT","Standard, Dist","Standard, FK","Standard, PK","Standard, PKatt","Expected, xG.1","Expected, npxG.1","Expected, npxG/Sh","Expected, G-xG","Expected, np:G-xG","Total, Cmp","Total, Att","Total, Cmp%","Total, TotDist","Total, PrgDist","Short, Cmp","Short, Att","Short, Cmp%","Medium, Cmp","Medium, Att","Medium, Cmp%","Long, Cmp","Long, Att","Long, Cmp%",Ast,xAG,"Expected, xA","Expected, A-xAG",KP,1/3,PPA,CrsPA,PrgP,Att,"Pass Types, Live","Pass Types, Dead","Pass Types, FK","Pass Types, TB","Pass Types, Sw","Pass Types, Crs","Pass Types, TI","Pass Types, CK","Corner Kicks, In","Corner Kicks, Out","Corner Kicks, Str","Outcomes, Cmp","Outcomes, Off","Outcomes, Blocks","SCA, SCA","SCA, SCA90","SCA Types, PassLive","SCA Types, PassDead","SCA Types, TO","SCA Types, Sh","SCA Types, Fld","SCA Types, Def","GCA, GCA","GCA, GCA90","GCA Types, PassLive","GCA Types, PassDead","GCA Types, TO","GCA Types, Sh","GCA Types, Fld","GCA Types, Def","Tackles, Tkl","Tackles, TklW","Tackles, Def 3rd","Tackles, Mid 3rd","Tackles, Att 3rd","Challenges, Tkl","Challenges, Att","Challenges, Tkl%","Challenges, Lost","Blocks, Blocks","Blocks, Sh","Blocks, Pass",Int,Tkl+Int,Clr,Err,"Touches, Touches","Touches, Def Pen","Touches, Def 3rd","Touches, Mid 3rd","Touches, Att 3rd","Touches, Att Pen","Touches, Live","Take-Ons, Att","Take-Ons, Succ","Take-Ons, Succ%","Take-Ons, Tkld","Take-Ons, Tkld%","Carries, Carries","Carries, TotDist","Carries, PrgDist","Carries, PrgC","Carries, 1/3","Carries, CPA","Carries, Mis","Carries, Dis","Receiving, Rec","Receiving, PrgR"
575,Aaron Cresswell-ENG-1989,2017-2018,Aaron Cresswell,eng ENG,1.0,West Ham,eng Premier League,27,1989,36,35,3069,34.1,1,3,4,1,0,0,7,0,0.8,0.8,3.2,3.9,35.0,128.0,62.0,0.03,0.09,0.12,0.03,0.12,0.02,0.09,0.12,0.02,0.12,36,3069,85,89.7,34.1,35,-999.0,30.0,1,-999.0,1,1.14,45.0,60.0,-15.0,-0.44,0.84,34.5,47.8,-13.3,-0.39,0.92,1,21.0,6,28.6,0.62,0.18,0.05,0.17,28.7,8.0,0,0,0.8,0.8,0.04,0.2,0.2,1204.0,1723.0,69.9,22518.0,9345.0,531.0,593.0,89.5,488.0,642.0,76.0,163.0,408.0,40.0,3,3.2,2.4,-0.2,34.0,135.0,21.0,10.0,128.0,1723.0,1346.0,361.0,82.0,3.0,23.0,196.0,212.0,67.0,43.0,14.0,2.0,1204.0,16.0,51.0,66.0,1.94,36.0,24.0,1.0,2.0,3.0,0.0,9.0,0.26,6.0,3.0,0.0,0.0,0.0,0.0,34.0,21.0,12.0,20.0,2.0,19.0,30.0,63.3,11.0,42.0,17.0,25.0,44.0,78.0,117.0,1.0,2035.0,120.0,572.0,982.0,497.0,15.0,2035.0,14.0,5.0,35.7,9.0,64.3,870.0,4032.0,1959.0,35.0,48.0,1.0,26.0,20.0,1076.0,62.0
3230,Aaron Cresswell-ENG-1989,2018-2019,Aaron Cresswell,eng ENG,1.0,West Ham,eng Premier League,28,1989,20,18,1589,17.7,0,1,1,0,0,0,1,0,0.5,0.5,1.1,1.5,35.0,94.0,103.0,0.00,0.06,0.06,0.00,0.06,0.03,0.06,0.09,0.03,0.09,20,1589,79,46.5,17.7,18,85.0,16.0,2,30.0,7,1.30,21.0,26.0,-5.0,-0.28,-0.38,19.7,25.2,-5.5,-0.31,0.12,0,11.0,0,0.

In [31]:
type(X_train)

numpy.ndarray

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

ss = StandardScaler()

ss.fit(X_train)

def sim(origin:list, other:list) -> float:
    """
    A function that takes in the stats of two players and returns how similar other is to origin
    origin and other should have length 163
    """
    O1 = ss.transform(origin)
    O2 = ss.transform(other)

    return cosine_similarity(vector1, vector2)

In [27]:
total.loc[total['Player'] == 'Mohamed Salah'].iloc[0, i_s]

Pos                               7.0
Age                                25
Born                             1992
Playing Time, MP                   36
Playing Time, Starts               34
Playing Time, Min                2907
Playing Time, 90s                32.3
Performance, Gls                   32
Performance, Ast                   10
Performance, G+A                   42
Performance, G-PK                  31
Performance, PK                     1
Performance, PKatt                  2
Performance, CrdY                   1
Performance, CrdR                   0
Expected, xG                     24.1
Expected, npxG                   22.6
Expected, xAG                     7.5
Expected, npxG+xAG               30.1
Progression, PrgC                95.0
Progression, PrgP                97.0
Progression, PrgR               393.0
Per 90 Minutes, Gls              0.99
Per 90 Minutes, Ast              0.31
Per 90 Minutes, G+A               1.3
Per 90 Minutes, G-PK             0.96
Per 90 Minut